In [3]:
import ee
import geemap
ee.Initialize()

In [5]:
Map = geemap.Map()

In [38]:
# Area of interest: Taounat,Morocco

AOI=ee.Geometry.Rectangle([-4.738979,34.511263,-4.540658,34.673896])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(AOI) \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]').clip(AOI)


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

Map.centerObject(AOI, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(bottom=416895.0, center=[34.5925930726126, -4.639818500001078], controls=(WidgetControl(options=['position…

In [13]:
training = image.sample(**{
    'region': AOI,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=417127.0, center=[34.57329875727302, -4.62627410888672], controls=(WidgetControl(options=['position…

In [32]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=208646.0, center=[34.60438909041995, -4.689788818359376], controls=(WidgetControl(options=['positio…

In [42]:
legend_keys = ['Fir forest (sapins)', 'Water', 'road/bare soil', 'arboriculture/reforestation', 'agriculture/habitat']
legend_colors = ['#00FF00', '#0000FF', '#BEBADA', '#FB8072','#000000']

#Reclassify the map
#result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

Map(bottom=833675.0, center=[34.616256875628956, -4.598979949951173], controls=(WidgetControl(options=['positi…

In [43]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))